## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Samarai,PG,-10.6167,150.6667,86.76,67,100,14.14,overcast clouds
1,1,Los Llanos De Aridane,ES,28.6585,-17.9182,56.95,77,20,6.91,few clouds
2,2,Nishihara,JP,26.1842,127.7558,76.68,97,100,24.16,mist
3,3,Morondava,MG,-20.2833,44.2833,77.67,90,8,5.59,clear sky
4,4,Oranjemund,NaN,-28.5500,16.4333,56.39,95,1,2.66,clear sky


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Samarai,PG,-10.6167,150.6667,86.76,67,100,14.14,overcast clouds
2,2,Nishihara,JP,26.1842,127.7558,76.68,97,100,24.16,mist
3,3,Morondava,MG,-20.2833,44.2833,77.67,90,8,5.59,clear sky
12,12,Janauba,BR,-15.8025,-43.3089,75.61,60,81,4.23,broken clouds
13,13,Saint George,US,37.1041,-113.5841,85.62,11,0,11.50,clear sky
16,16,Puerto Ayora,EC,-0.7393,-90.3518,77.81,85,54,5.14,light rain
22,22,Arraial Do Cabo,BR,-22.9661,-42.0278,76.96,94,0,11.50,clear sky
31,31,Rikitea,PF,-23.1203,-134.9692,79.81,72,15,16.42,light rain
34,34,Bambous Virieux,MU,-20.3428,57.7575,77.22,94,40,4.61,light rain
36,36,Manadhoo,MV,5.7667,73.3833,81.12,66,0,3.38,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                192
City                   192
Country                192
Lat                    192
Lng                    192
Max Temp               192
Humidity               192
Cloudiness             192
Wind Speed             192
Current Description    192
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Samarai,PG,86.76,overcast clouds,-10.6167,150.6667,
2,Nishihara,JP,76.68,mist,26.1842,127.7558,
3,Morondava,MG,77.67,clear sky,-20.2833,44.2833,
12,Janauba,BR,75.61,broken clouds,-15.8025,-43.3089,
13,Saint George,US,85.62,clear sky,37.1041,-113.5841,
16,Puerto Ayora,EC,77.81,light rain,-0.7393,-90.3518,
22,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,
31,Rikitea,PF,79.81,light rain,-23.1203,-134.9692,
34,Bambous Virieux,MU,77.22,light rain,-20.3428,57.7575,
36,Manadhoo,MV,81.12,clear sky,5.7667,73.3833,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [9]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df[hotel_df['Hotel Name'] != '']
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Samarai,PG,86.76,overcast clouds,-10.6167,150.6667,Nuli Sapi
2,Nishihara,JP,76.68,mist,26.1842,127.7558,Minshuku Agaihama
3,Morondava,MG,77.67,clear sky,-20.2833,44.2833,La Case Bambou
12,Janauba,BR,75.61,broken clouds,-15.8025,-43.3089,Ouro Norte Palace Hotel
13,Saint George,US,85.62,clear sky,37.1041,-113.5841,Best Western Plus Abbey Inn
16,Puerto Ayora,EC,77.81,light rain,-0.7393,-90.3518,Finch Bay Galapagos Hotel
22,Arraial Do Cabo,BR,76.96,clear sky,-22.9661,-42.0278,Pousada Porto Praia
31,Rikitea,PF,79.81,light rain,-23.1203,-134.9692,People ThankYou
34,Bambous Virieux,MU,77.22,light rain,-20.3428,57.7575,Casa Tia Villa
36,Manadhoo,MV,81.12,clear sky,5.7667,73.3833,Bon Abri maldives


In [10]:
# 8a. Create the output File (CSV)
output_data_file = '../Vacation_Search/WeatherPy_Vacation.csv'

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [11]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [12]:
# 11a. Add a marker layer for each city to the map.
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))